### Day 21

### Part 1:
- Dirac Dice:
    - Circular track with 10 spaces (1-10, not zero)
    - Random starting space
    - On your turn:
        - Roll 3 times and add up the result
        - Move that many spaces forward
        - Increase score by the number on the board
        - First player to reach 1000 wins
   - But we're using a deterministic die that rolls 1, 2, 3, etc. up to 100.
   - Answer is losing score * number of dice rolls
   
- Thoughts:
    - Can probably do this with modulo arithmetic and not have to iterate, but that might be more difficult.
    - So just do a class and iterate unless we can't do it for part 2

In [1]:
class DeterministicDiceGame(object):
    def __init__(self,starting_pos):
        # Positions
        self.p1 = starting_pos[0]
        self.p2 = starting_pos[1]
        
        # Scores
        self.p1_score = 0
        self.p2_score = 0
        
        self.next_turn = "p1"
        self.rolls = 0
        self.last_roll = 0
        self.winning_score = 1000
        
        self.next_turn_dict = {"p1":"p2","p2":"p1"}
        
    def roll(self):
        """Roll the dice and return the value."""
        self.last_roll = (self.last_roll % 100) + 1
        self.rolls += 1
        return self.last_roll
    
    def roll_3_dice(self):
        """Roll 3 dice and return the value."""
        r1 = self.roll()
        r2 = self.roll()
        r3 = self.roll()
        return r1 + r2 + r3
        
    def one_turn(self):
        """Simulate the turn of each player."""
        # Roll 3 dice
        total_roll = self.roll_3_dice()
        
        # Move pawn
        if self.next_turn == "p1":
            # Need to do this because 10%10 == 0, but spaces are 1-10
            self.p1 = ((self.p1 + total_roll -1) % 10) +1
            self.p1_score += self.p1
        else:
            self.p2 = ((self.p2 + total_roll -1) % 10) +1
            self.p2_score += self.p2
            
        if self.p1_score >= self.winning_score:
            print("P1 wins!")
            self.losing_score = self.p2_score
            return True
        elif self.p2_score >= self.winning_score:
            print("P2 wins!")
            self.losing_score = self.p1_score
            return True
        else:
            self.next_turn = self.next_turn_dict[self.next_turn]
            return False
        
    def play_until_winner(self):
        winner = False
        
        while not winner:
            winner = self.one_turn()
            
        return self.losing_score * self.rolls

In [2]:
# Test input
d = DeterministicDiceGame([4,8])
d.play_until_winner()

P1 wins!


739785

In [3]:
# Puzzle input
d = DeterministicDiceGame([4,6])
d.play_until_winner()

P1 wins!


888735

### Part 2:
- Each time you roll, universe splits into 3: where you roll 1,2 and 3 respectively
- But win at 21 instead of 1000
- So for each turn, 3*3*3 = 27 different outcomes. 
- But really you can only roll numbers between 3 - 9 (7 outcomes with different weightings)
    - This would mean the number of universes with different outcomes is 7^turns, compared to 27^turns if we simulated them individually
- Thoughts:
    - Clearly can't brute force
    - Will need a recursive function that counts number of winners from a given game state
    - There will be a lot of duplicates since we might end up in the same game state. 
        - Can probably cache solutions defined by: current player score + position, other player score + position

In [4]:
from functools import lru_cache
vals = [3,4,5,6,7,8,9] # Possible rolls
weights = [1,3,6,7,6,3,1] # Number of ways to get each one

@lru_cache(maxsize=None)
def play_until_winner(p1_score,p1_pos,p2_score,p2_pos):
    """
    Simulate all possible outcomes of the game until we have a winner
    
    p1_score: Score of player who just rolled
    p1_pos: Position of player who just rolled
    p2_score: Score of player whose turn is next
    p2_pos: Position of player whose turn is next
    
    Returns:
    number of times each player wins:
        (active_player, other_player)
    """
    
    # Did they win?
    if p1_score >= 21:
        return (1,0)
    
    # Otherwise simulate all of the outcomes for the next player's roll
    all_p1_wins = 0
    all_p2_wins = 0
    for ix,v in enumerate(vals):
        
        new_pos = ((p2_pos + v -1) % 10) +1
        new_score = p2_score + new_pos
        
        # How many times does each player win from here?
        p2_wins, p1_wins = play_until_winner(new_score, new_pos,p1_score,p1_pos)
        
        all_p1_wins += p1_wins * weights[ix]
        all_p2_wins += p2_wins * weights[ix]
        
    return all_p1_wins, all_p2_wins

In [5]:
# Test input:
print(play_until_winner(21,4,0,8)) # P1 already won, should be (1,0)
print(play_until_winner(5,4,20,8)) # P2 wins every time, should be 27 for p2
play_until_winner(0,8,0,4)

(1, 0)
(0, 27)


(341960390180808, 444356092776315)

In [6]:
# Puzzle input
play_until_winner(0,6,0,4)

(447445126742668, 647608359455719)